# This is a POC notebook:
#### The purpose of this notebook is to perform document embedding using the weighted average of the embedding at the sentence level.

## Import libraries

In [23]:
import sys

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

from sem_covid.services.data_registry import Dataset
from sem_covid.services.model_registry import EmbeddingModelRegistry
import numpy as np
from sem_covid.services.sc_wrangling.sentences_ranker import textual_tfidf_ranker

## Download textual dataset

In [24]:
pwdb = Dataset.PWDB.fetch()[['background_info_description', 'content_of_measure_description']]

100% (1368 of 1368) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


## Select document

In [4]:
document = pwdb['content_of_measure_description'][0]

## Split in sentences

In [5]:
sentences = document.split('.')

## Compute sentences weights

In [6]:
sentence_weights = textual_tfidf_ranker(sentences, top_k=10)

## Load embedding model

In [7]:
emb_model = EmbeddingModelRegistry().sent2vec_universal_sent_encoding()

INFO:absl:Using /tmp/tfhub_modules to cache modules.
2021-09-07 14:21:39.654981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-07 14:21:39.655002: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-07 14:21:39.655021: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c004c8213d64): /proc/driver/nvidia/version does not exist
2021-09-07 14:21:39.655134: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-07 14:21:41.174374: I tensorflow/compiler/mlir/ml

## Compute embeddings for sentences

In [8]:
sentence_embeddings = emb_model.encode(sentences)

## Compute weighted embedding for initial document

In [9]:
document_embedding = np.average(sentence_embeddings, axis=0, weights=sentence_weights)

## Show document embedding and check vector dimension

In [10]:
document_embedding

array([-1.93257768e-02, -1.40374574e-02,  2.06236099e-02,  1.38837699e-02,
        1.65766544e-02, -1.95122308e-03, -1.85330101e-03, -9.71754352e-03,
        4.45727734e-02, -5.61452577e-02, -3.05634340e-02,  2.47388971e-02,
       -2.50115270e-02,  1.20743941e-02, -1.82846570e-03, -1.84467986e-02,
       -3.58205181e-02,  2.33170658e-02, -1.24824259e-02,  2.34369861e-02,
       -4.50854156e-02, -1.53129495e-03,  3.10274502e-02,  1.20375021e-02,
        1.63653498e-02,  5.44248223e-02, -5.74489471e-02,  1.89698847e-02,
       -1.18967976e-02, -1.50139499e-04,  2.22514893e-02,  6.14701657e-03,
        2.88664795e-02,  2.82122060e-02, -3.03304515e-02, -2.34066825e-02,
        2.08173459e-02, -2.62518656e-02,  6.31360541e-03, -3.17640993e-02,
       -1.34031136e-02, -1.13748790e-03, -1.75902641e-02,  1.57720214e-02,
        2.76829824e-03,  3.60610125e-03,  3.04531006e-02,  2.25567331e-02,
        1.34938536e-03, -2.52939031e-02, -2.91051485e-02,  2.09360992e-02,
       -3.56091379e-03,  

In [13]:
len(document_embedding) == 512



True